# Overview
contains simple queries to verify the statistics of the synthetic dataset generated


In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
# Connect to your DuckDB database
dbpath = "/Users/omid/PycharmProjects/benchmarker_omid/_output/data/varchars_grp_size_zipf=1.0_nrows=20.0M_uniques=1000000_len=32/varchars-grp-size-1000000.db"
con = duckdb.connect(dbpath)

# Load your table
table_name = "varchars"

## Basic statistics
Number of rows and columns
Unique cardinality of each column
Min, Max, Avg length of each column 

In [ ]:
num_rows = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
num_columns = len(con.execute(f"PRAGMA table_info('{table_name}')").fetchall())

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}\n")

columns = con.execute(f"PRAGMA table_info('{table_name}')").df()['name'].tolist()

col_stats = []

for col in columns:
    query = f"""
    SELECT 
        COUNT(DISTINCT {col}) AS unique_count,
        AVG(LENGTH({col})) AS avg_length,
        MIN(LENGTH({col})) AS min_length,
        MAX(LENGTH({col})) AS max_length
    FROM {table_name}
    WHERE {col} IS NOT NULL
    """
    result = con.execute(query).fetchdf()
    result.insert(0, "column", col)
    col_stats.append(result)

import pandas as pd
stats_df = pd.concat(col_stats, ignore_index=True)

print("Column Statistics:\n", stats_df, "\n")

## Frequency distribution
Plots the top k most frequent values. 
It shows the distribution of values within the dataset

In [ ]:
top_k = 10000

# Get all column names
columns = con.execute(f"PRAGMA table_info('{table_name}')").fetchdf()["name"].tolist()

# Loop over each column
for col in columns:
    print(f"Processing column: {col}")
    
    query = f"""
        SELECT {col},COUNT(*) AS freq
        FROM {table_name}
        WHERE {col} IS NOT NULL
        GROUP BY {col}
        ORDER BY freq DESC
        LIMIT {top_k}
    """

    freq_df = con.execute(query).fetchdf()

    if freq_df.empty:
        print(f"  Skipping column '{col}' (no non-null values)")
        continue

    # Plot rank-frequency curve
    plt.figure(figsize=(10, 4))
    plt.plot(range(1, len(freq_df) + 1), freq_df['freq'])
    plt.yscale('log')  # Helps with skewed frequencies
    plt.title(f"Top {top_k} Value Frequencies in '{col}'")
    plt.xlabel("Rank (1 = Most Frequent)")
    plt.ylabel("Frequency (log scale)")
    plt.tight_layout()
    plt.show()
    
    print(freq_df.head(100))
    freq_df
    

## Unique values within each column
Double checking that each column contain unique strings not available in other columns

In [ ]:
columns = con.execute(f"PRAGMA table_info('{table_name}')").fetchdf()["name"].tolist()

# Safety: drop the temp table if it exists
con.execute("DROP TABLE IF EXISTS unique_strs")

# Create a new table for accumulating unique strings
con.execute("CREATE TABLE unique_strs (value TEXT)")

# Insert DISTINCT values from each column (skip NULLs)
for col in columns:
    print(f"Inserting unique values from column: {col}")
    con.execute(f"""
        INSERT INTO unique_strs
        SELECT DISTINCT {col}
        FROM {table_name}
        WHERE {col} IS NOT NULL
    """)

# Check for duplicates — values that appeared in more than one column
dup_query = """
    SELECT value, COUNT(*) AS occurrences
    FROM unique_strs
    GROUP BY value
    HAVING COUNT(*) > 1
    ORDER BY occurrences DESC
    LIMIT 100
"""

duplicates_df = con.execute(dup_query).fetchdf()
duplicates_df

## Compression Analysis 
The following script will analyze the number of segments related to each column, the different types of compression used for those columns. 

In [ ]:
# Get the storage_info table
df = con.execute(f"select * from pragma_storage_info('{table_name}') where segment_type = 'VARCHAR'").fetch_df()


In [ ]:
# Get the storage_info table
df = con.execute(f"select * from pragma_storage_info('{table_name}') where segment_type = 'VARCHAR'").fetch_df()

# Group and aggregate the data
summary_df = (
    df.groupby(['column_name', 'compression'], as_index=False)
    .agg(
        total_count=('count', 'sum'),
        num_segments=('count', 'count'),
        avg_count=('count', 'mean')
    )
    .sort_values(['column_name', 'total_count'], ascending=[True, False])
)

# Display the result
print(summary_df)
